In [0]:
! wget https://loto6.thekyo.jp/data/loto6.csv

--2020-01-16 06:38:27--  https://loto6.thekyo.jp/data/loto6.csv
Resolving loto6.thekyo.jp (loto6.thekyo.jp)... 133.242.141.109
Connecting to loto6.thekyo.jp (loto6.thekyo.jp)|133.242.141.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138006 (135K) [application/octet-stream]
Saving to: ‘loto6.csv’

loto6.csv           100%[===================>] 134.77K   107KB/s    in 1.3s    

2020-01-16 06:38:30 (107 KB/s) - ‘loto6.csv’ saved [138006/138006]



In [0]:
import pandas as pd
data=pd.read_csv("loto6.csv",encoding="SHIFT-JIS")
data.head()

,開催回,日付,第1数字,第2数字,第3数字,第4数字,第5数字,第6数字,BONUS数字,1等口数,2等口数,3等口数,4等口数,5等口数,1等賞金,2等賞金,3等賞金,4等賞金,5等賞金,キャリーオーバー
0,1,2000/10/5,2,8,10,13,27,30,39,2,2,262,12413,174452,45513600,40961900,375200,6900,1000,0
1,2,2000/10/12,1,9,16,20,21,43,5,0,7,195,8426,144860,0,13188800,568100,11500,1000,102580607
2,3,2000/10/19,1,5,15,31,36,38,13,2,8,164,9450,181437,121348100,15762800,922700,14100,1000,0
3,4,2000/10/26,16,18,26,27,34,40,13,0,3,134,8675,160892,0,38645500,1038200,14100,1000,128819603
4,5,2000/11/2,9,15,21,23,27,28,43,0,9,258,16473,239987,0,13516100,565700,7800,1000,263982587


In [0]:
data_2018=data.query(" \"2011/01/01\"< 日付 < \"2018/12/31\" ")
data_latest_year=data.query('\"2019/01/01\"<日付<\"2020/12/31\"')

In [0]:
dataset=data_2018.loc[:,["第1数字","第2数字","第3数字","第4数字","第5数字","第6数字",]]
data_latest_year=data_latest_year.loc[:,["第1数字","第2数字","第3数字","第4数字","第5数字","第6数字",]]
dataset.head()
dataset=dataset.values
latest=data_latest_year.values

In [0]:
import numpy as np
import copy
def int2List(inputint):
  b=bin(inputint)
  b=b[2::]
  while len(b)<6:
    b="0"+b
  
  return [c for c in b]
def createData(arr:np.ndarray,rate:float,test=True):
    trainX=[]
    x=[]
    list_arr=arr.tolist()
    for l in list_arr[0::]:
      x.clear()
      for i in l:
        x.extend(int2List(i))
      trainX.append(x)
    trainY=copy.deepcopy(trainX)  
    trainlen=int(len(trainX)*rate)  ## 1 2 3 4 
                                    ## 2 3 4
    trainX=np.array(trainX,dtype="float64")
    trainY=np.array(trainY,dtype="float64")
    if test:
      return trainX[0:trainlen-1],trainY[1:trainlen],trainX[trainlen-1:len(trainX)-1:],trainY[trainlen::]
    else : 
      return trainX[:len(trainX)-1], trainY[1:]

In [0]:
x_test,y_test,x_train,y_train=createData(dataset,0.3)   
latest_x_train,latest_y_train=createData(latest,0,False)   
print(len(x_train),len(y_train),len(x_test),len(y_test))
print(len(latest_x_train),len(latest_y_train))

521 521 221 221
104 104


In [0]:
a=[1,2,3,4,5]
print(a[:len(a)-1:])
print(a[1:])

[1, 2, 3, 4]
[2, 3, 4, 5]


In [0]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K

batch_size = 32
## 1~43(6bit) * 6
num_classes = 6 * 6
epochs = 800
  
 
model = Sequential()
model.add(Dense(2000,activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(5000, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(5000, activation='relu'))
model.add(Dense(1000,activation="relu"))
model.add(Dense(1000, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(num_classes, activation='relu'))

 
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [0]:

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(latest_x_train, latest_y_train))
score = model.evaluate(x_test, y_test, verbose=0)


Train on 521 samples, validate on 104 samples
Epoch 1/800
521/521 [==============================] - 0s 792us/sample - loss: 3.9259 - acc: 0.6111 - val_loss: 4.5434 - val_acc: 0.6111
Epoch 2/800
521/521 [==============================] - 0s 306us/sample - loss: 3.4531 - acc: 0.6111 - val_loss: 4.2684 - val_acc: 0.6111
Epoch 3/800
521/521 [==============================] - 0s 294us/sample - loss: 3.0046 - acc: 0.6111 - val_loss: 4.2173 - val_acc: 0.6111
Epoch 4/800
521/521 [==============================] - 0s 289us/sample - loss: 2.6622 - acc: 0.6111 - val_loss: 4.1780 - val_acc: 0.6111
Epoch 5/800
521/521 [==============================] - 0s 295us/sample - loss: 2.4228 - acc: 0.6111 - val_loss: 4.1574 - val_acc: 0.6111
Epoch 6/800
521/521 [==============================] - 0s 300us/sample - loss: 2.2678 - acc: 0.6111 - val_loss: 3.8974 - val_acc: 0.6111
Epoch 7/800
521/521 [==============================] - 0s 293us/sample - loss: 2.1127 - acc: 0.6111 - val_loss: 3.8721 - val_acc: 0.

In [0]:
model.evaluate(latest_x_train,latest_y_train)